# Машинное обучение 1: линейная регрессия

### Часть 1. 
##### Научимся строить модель линейной регрессии для больших объемов данных и оценивать ее качество

### Задание

Разделите набор данных на обучающие/проверочные в пропорции 80/20.

Загрузите данные и очистите значения (нулями и средними). Постройте модель линейной регрессии для каждого часа в отдельности, используя температуру воздуха (air_temperature), влажность (dew_temperature), атмосферное давление (sea_level_pressure), скорость ветра (wind_speed) и облачность (cloud_coverage).

Рассчитайте качество построенной модели по проверочным данным. Используйте данные:

http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz

http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz

http://video.ittensive.com/machine-learning/ashrae/train.0.0.csv.gz

Какое получилось качество модели линейной регрессии по часам с точностью до десятых?

_______________________________________________________________________________________________________________________________

Импортируем библиотеки.

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

Загружаем данные в датафреймы.

In [2]:
df_buildings = pd.read_csv('http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz')
df_weather = pd.read_csv('http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz').set_index(['timestamp', 'site_id'])
df_train = pd.read_csv('http://video.ittensive.com/machine-learning/ashrae/train.0.0.csv.gz')

print(df_buildings.head())

   site_id  building_id primary_use  square_feet  year_built  floor_count
0        0            0   Education         7432      2008.0          NaN
1        0            1   Education         2720      2004.0          NaN
2        0            2   Education         5376      1991.0          NaN
3        0            3   Education        23685      2002.0          NaN
4        0            4   Education       116607      1975.0          NaN


In [3]:
print(df_weather.head())

                             air_temperature  cloud_coverage  dew_temperature  \
timestamp           site_id                                                     
2016-01-01 00:00:00 0                   25.0             6.0             20.0   
2016-01-01 01:00:00 0                   24.4             NaN             21.1   
2016-01-01 02:00:00 0                   22.8             2.0             21.1   
2016-01-01 03:00:00 0                   21.1             2.0             20.6   
2016-01-01 04:00:00 0                   20.0             2.0             20.0   

                             precip_depth_1_hr  sea_level_pressure  \
timestamp           site_id                                          
2016-01-01 00:00:00 0                      NaN              1019.7   
2016-01-01 01:00:00 0                     -1.0              1020.2   
2016-01-01 02:00:00 0                      0.0              1020.2   
2016-01-01 03:00:00 0                      0.0              1020.1   
2016-01-01 0

In [4]:
print(df_train.head())

   building_id  meter            timestamp  meter_reading
0            0      0  2016-01-01 00:00:00            0.0
1            0      0  2016-01-01 01:00:00            0.0
2            0      0  2016-01-01 02:00:00            0.0
3            0      0  2016-01-01 03:00:00            0.0
4            0      0  2016-01-01 04:00:00            0.0


Объединяем датафреймы в один по общим колонкам.

In [5]:
df = df_train.merge(df_buildings, how = 'left', on = 'building_id')
df.set_index(['timestamp', 'site_id'], inplace = True)
df = df.merge(df_weather, left_index = True, right_index = True, how = 'left')
df.reset_index(inplace = True)
df.head()

,timestamp,site_id,building_id,meter,meter_reading,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,2016-01-01 00:00:00,0,0,0,0.0,Education,7432,2008.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,2016-01-01 01:00:00,0,0,0,0.0,Education,7432,2008.0,NaN,24.4,NaN,21.1,-1.0,1020.2,70.0,1.5
2,2016-01-01 02:00:00,0,0,0,0.0,Education,7432,2008.0,NaN,22.8,2.0,21.1,0.0,1020.2,0.0,0.0
3,2016-01-01 03:00:00,0,0,0,0.0,Education,7432,2008.0,NaN,21.1,2.0,20.6,0.0,1020.1,0.0,0.0
4,2016-01-01 04:00:00,0,0,0,0.0,Education,7432,2008.0,NaN,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6


Проверяем наличие пропусков и типы данных.

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8784 entries, 0 to 8783
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   timestamp           8784 non-null   object 
 1   site_id             8784 non-null   int64  
 2   building_id         8784 non-null   int64  
 3   meter               8784 non-null   int64  
 4   meter_reading       8784 non-null   float64
 5   primary_use         8784 non-null   object 
 6   square_feet         8784 non-null   int64  
 7   year_built          8784 non-null   float64
 8   floor_count         0 non-null      float64
 9   air_temperature     8781 non-null   float64
 10  cloud_coverage      4954 non-null   float64
 11  dew_temperature     8781 non-null   float64
 12  precip_depth_1_hr   8783 non-null   float64
 13  sea_level_pressure  8699 non-null   float64
 14  wind_direction      8534 non-null   float64
 15  wind_speed          8784 non-null   float64
dtypes: flo

Приводим дату к типу datetime и выделяем данные по часам в отдельную колонку.

In [7]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['hours'] = df['timestamp'].apply(lambda x: x.hour)
df.head()

,timestamp,site_id,building_id,meter,meter_reading,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,hours
0,2016-01-01 00:00:00,0,0,0,0.0,Education,7432,2008.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,0
1,2016-01-01 01:00:00,0,0,0,0.0,Education,7432,2008.0,NaN,24.4,NaN,21.1,-1.0,1020.2,70.0,1.5,1
2,2016-01-01 02:00:00,0,0,0,0.0,Education,7432,2008.0,NaN,22.8,2.0,21.1,0.0,1020.2,0.0,0.0,2
3,2016-01-01 03:00:00,0,0,0,0.0,Education,7432,2008.0,NaN,21.1,2.0,20.6,0.0,1020.1,0.0,0.0,3
4,2016-01-01 04:00:00,0,0,0,0.0,Education,7432,2008.0,NaN,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6,4


Заполняем пропуски данных средними значениями.

In [8]:
means = []
columns = ['air_temperature', 'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure', 'wind_direction']
for col in columns:
    means.append(df[col][df[col].notnull()].mean())
    df[col].fillna('no_value', inplace = True)
for m, col in zip(means, columns):
    df[col] = df[col].apply(lambda x: m if x == 'no_value' else x)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8784 entries, 0 to 8783
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   timestamp           8784 non-null   datetime64[ns]
 1   site_id             8784 non-null   int64         
 2   building_id         8784 non-null   int64         
 3   meter               8784 non-null   int64         
 4   meter_reading       8784 non-null   float64       
 5   primary_use         8784 non-null   object        
 6   square_feet         8784 non-null   int64         
 7   year_built          8784 non-null   float64       
 8   floor_count         0 non-null      float64       
 9   air_temperature     8784 non-null   float64       
 10  cloud_coverage      8784 non-null   float64       
 11  dew_temperature     8784 non-null   float64       
 12  precip_depth_1_hr   8784 non-null   float64       
 13  sea_level_pressure  8784 non-null   float64     

Обучаем модели линейной регрессии по часам. Сохраняем коэффициенты получившихся моделей в список.

In [9]:
data = df[['hours', 'air_temperature', 'dew_temperature', 'sea_level_pressure', 'wind_speed', 'cloud_coverage', 'meter_reading']]
coeffs = []
errors = []
for hour in data.hours.unique():
    temp_data = data[data.hours == hour]
    temp_data = temp_data[temp_data.meter_reading > 0]
    X = temp_data[['air_temperature', 'dew_temperature', 'sea_level_pressure', 'wind_speed', 'cloud_coverage']]
    y = temp_data['meter_reading']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
    model = LinearRegression()
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    errors.append(mean_squared_error(y_test, preds))
    coeffs.append(model.coef_)
    coeffs[hour] = np.append(coeffs[hour], model.intercept_)

Разделяем общий датафрейм на тренировочный и проверочный.

In [10]:
data = data[data['meter_reading'] > 0]
data_train, data_test = train_test_split(data, test_size = 0.2, random_state = 42)
data_test.head()

,hours,air_temperature,dew_temperature,sea_level_pressure,wind_speed,cloud_coverage,meter_reading
3663,15,27.8,21.1,1017.0,1.5,2.0,253.9120
5103,15,31.1,22.8,1019.0,1.5,2.0,292.8180
7609,1,19.4,11.7,1018.9,2.1,4.0,161.7670
8244,12,10.6,6.7,1025.1,7.7,8.0,95.5584
4446,6,26.1,23.9,1019.9,1.5,2.0,252.5470


In [17]:
coeffs[0]

array([ 5.21654766e+00,  2.24701873e+00, -6.98547268e-01, -3.31444424e+00,
       -1.46366053e+00,  7.93856523e+02])

Вычисляем качество почасовой модели.

In [16]:
def calculate_model(x):
    model = coeffs[int(x.hours)]
    meter_reading_log = np.log(x.meter_reading + 1)
    meter_reading_lr = np.log(1 + x.air_temperature * model[0] + 
        x.dew_temperature * model[1] + x.sea_level_pressure * model[2] +
        x.wind_speed * model[3] + x.cloud_coverage * model[4] + model[5])
    x['meter_reading_lr_q'] = (meter_reading_log - meter_reading_lr)**2
    return x

data_test = data_test.apply(calculate_model, axis=1, result_type='expand')
data_test_lr_rmsle = np.sqrt(data_test['meter_reading_lr_q'].sum() / len(data_test))
print (f'Качество почасовой линейной регрессии, 5 параметров: {round(data_test_lr_rmsle, 1)}')

Качество почасовой линейной регрессии, 5 параметров: 0.2


### Часть 2
##### Научимся разделять данные на отдельные части, заполнять пропуски в данных и оптимизировать потребление памяти

### Задание

Получите данные по энергопотреблению первых 20 зданий (building_id от 0 до 19).

Заполните отсутствующие значения по погоде интерполяционными данными.

Разделите данные на обучающие/проверочные в пропорции 80/20.

Постройте и найдите общее качество модели линейной регрессии, построенной по часам для каждого из первых 20 зданий по следующим параметрам: air_temperature, dew_temperature, cloud_coverage, wind_speed, precip_depth_1_hr, sea_level_pressure, is_holiday.

Всего требуется построить 480 моделей линейной регрессии, вычислить по ним проверочные значения энергопотребления и получить итоговую оценку качества такой модели.

Для расчета последнего параметра (is_holiday) используйте график публичных выходных в США: USFederalHolidayCalendar

Исходные данные:

https://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz

https://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz

https://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz

Чему равна метрика качества полученной модели с точностью до десятых?

_______________________________________________________________________________________________________________________________

Импортируем библиотеки.

In [65]:
import pandas as pd
import numpy as np
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

Загружаем данные в датафреймы и фильтруем по первым 20 зданиям.

In [66]:
df_buildings = pd.read_csv('http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz')
df_weather = pd.read_csv('http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz').set_index(['timestamp', 'site_id'])
df_energy = pd.read_csv('http://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz')
df_energy = df_energy[df_energy['building_id'].isin(list(range(0, 20)))]

Соединяем датафреймы по индексам. Приводим дату к типу datetime, затем создаём колонку с часами.

In [67]:
df = df_energy.merge(df_buildings, how = 'left', on = 'building_id')
df.set_index(['timestamp', 'site_id'], inplace = True)
df = df.merge(df_weather, left_index = True, right_index = True, how = 'left')
df.reset_index(inplace = True)
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['hours'] = df['timestamp'].apply(lambda x: x.hour)
df = df.drop(columns=["meter", "site_id", "floor_count"], axis=1)
df.head()

,timestamp,building_id,meter_reading,primary_use,square_feet,year_built,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,hours
0,2016-01-01,0,0.0,Education,7432,2008.0,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,0
1,2016-01-01,1,0.0,Education,2720,2004.0,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,0
2,2016-01-01,2,0.0,Education,5376,1991.0,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,0
3,2016-01-01,3,0.0,Education,23685,2002.0,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,0
4,2016-01-01,4,0.0,Education,116607,1975.0,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,0


In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175680 entries, 0 to 175679
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   timestamp           175680 non-null  datetime64[ns]
 1   building_id         175680 non-null  int64         
 2   meter_reading       175680 non-null  float64       
 3   primary_use         175680 non-null  object        
 4   square_feet         175680 non-null  int64         
 5   year_built          175680 non-null  float64       
 6   air_temperature     175620 non-null  float64       
 7   cloud_coverage      99080 non-null   float64       
 8   dew_temperature     175620 non-null  float64       
 9   precip_depth_1_hr   175660 non-null  float64       
 10  sea_level_pressure  173980 non-null  float64       
 11  wind_direction      170680 non-null  float64       
 12  wind_speed          175680 non-null  float64       
 13  hours               175680 no

Функция уменьщения потребления памяти. Итерирует по колонкам, проверяет их тип и, если возможно, приводит их к типу, потребляющему меньше памяти.

In [69]:
def reduce_mem_usage (df):
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if str(col_type)[:5] == 'float':
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo('f2').min and c_max < np.finfo('f2').max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo('f4').min and c_max < np.finfo('f4').max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
        elif str(col_type)[:3] == 'int':
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo('i1').min and c_max < np.iinfo('i1').max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo('i2').min and c_max < np.iinfo('i2').max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo('i4').min and c_max < np.iinfo('i4').max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo('i8').min and c_max < np.iinfo('i8').max:
                df[col] = df[col].astype(np.int64)
        elif col == 'timestamp':
            df[col] = pd.to_datetime(df[col])
        elif str(col_type)[:8] != 'datetime':
            df[col] = df[col].astype('category')
    end_mem = df.memory_usage().sum() / 1024**2
    print(f'Потребление памяти меньше на {round(start_mem - end_mem, 2)} Мб (минус {round(100 * (start_mem - end_mem) / start_mem, 1)} %)')
    return df

In [70]:
df = reduce_mem_usage(df)

Потребление памяти меньше на 13.24 Мб (минус 70.5 %


Заполняем пропуски интерполяцией данных.

In [71]:
df['precip_depth_1_hr'] = df['precip_depth_1_hr'].apply(lambda x: x if x > 0 else 0)
interpolate_columns = ['air_temperature', 'dew_temperature', 'cloud_coverage', 
                       'wind_speed', 'precip_depth_1_hr', 'sea_level_pressure']
for col in interpolate_columns:
    df[col] = df[col].interpolate(limit_direction='both', kind='cubic')
    
pd.set_option('use_inf_as_na', True)
for col in interpolate_columns:
    print(col, "Inf+NaN:", df[col].isnull().sum())

air_temperature Inf+NaN: 0
dew_temperature Inf+NaN: 0
cloud_coverage Inf+NaN: 0
wind_speed Inf+NaN: 0
precip_depth_1_hr Inf+NaN: 0
sea_level_pressure Inf+NaN: 0


Создаём колонку поясняющую выходной день или нет.

In [72]:
dates_range = pd.date_range(start = '2015-12-31', end = '2017-01-01')
us_holidays = calendar().holidays(start = dates_range.min(), end = dates_range.max())
df['is_holiday'] = (df['timestamp'].isin(us_holidays)).astype('int8')

Фильтруем данные по показаниям счётчика, они должны быть больше нуля. Делим данные на тестовые и тренировочные.

In [73]:
df = df[['building_id', 'hours', 'air_temperature', 'dew_temperature', 'cloud_coverage', 'wind_speed',
        'precip_depth_1_hr', 'sea_level_pressure', 'is_holiday', 'meter_reading']][df.meter_reading > 0]
df_train, df_test = train_test_split(df, test_size = 0.2, random_state = 42)

Итерируем по индексу зданий и часам. Для каждого здания создаём почасовую модель потребления электричества. Заносим полученные коэффициенты в список.

In [91]:
models = []
coeffs = []
for build_id in sorted(df['building_id'].unique()):
    for hour in sorted(df.hours.unique()):
        X = df_train[df_train['building_id'] == build_id].drop(columns = ['building_id', 'meter_reading'])
        X = X[X['hours'] == hour].drop(columns = 'hours')
        y = df_train[df_train['building_id'] == build_id]
        y = y[y['hours'] == hour]['meter_reading']
        model = LinearRegression()
        model.fit(X, y)
        coeffs.append(model.coef_)
        coeffs[hour] = np.append(coeffs[hour], model.intercept_)
    models.append(coeffs)
    coeffs = []

Проверяем качество модели.

In [94]:
def calculate_model(x):
    model = models[int(x.building_id)][int(x.hours)]
    meter_reading_log = np.log(x.meter_reading + 1)
    meter_reading_lr = np.log(1 + x.air_temperature * model[0] + 
        x.dew_temperature * model[1] + x.cloud_coverage * model[2] +
        x.wind_speed * model[3] + x.precip_depth_1_hr * model[4] + x.sea_level_pressure * model[5] + 
        x.is_holiday * model[6] + model[7])
    x['meter_reading_lr_q'] = (meter_reading_log - meter_reading_lr)**2
    return x

df_test = df_test.apply(calculate_model, axis=1, result_type='expand')
df_test_lr_rmsle = np.sqrt(df_test['meter_reading_lr_q'].sum() / len(df_test))
print (f'Качество модели линейной регрессии: {round(df_test_lr_rmsle, 6)}')

D:\Anaconda\envs\AI\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in log
  import sys


Качество модели линейной регрессии: 0.246095


### Часть 3
##### Сравним предсказательную силу нескольких частных моделей линейной регрессии

### Задание

Получите данные по энергопотреблению первых 20 зданий (building_id от 0 до 19).

Заполните отсутствующие значения по погоде интерполяционными данными.

Разделите данные на обучающие/проверочные в пропорции 80/20.

Постройте (1) первый набор моделей линейной регрессии по часам для каждого из первых 20 зданий по следующим параметрам: air_temperature, dew_temperature, cloud_coverage, wind_speed, sea_level_pressure.

Постройте для этих же 20 зданий (2) второй набор моделей линейной регрессии по часам по параметрам: дни недели и праздники (is_holiday). Требуется построить еще 480 моделей.

Используйте логарифм целевого показателя (meter_reading_log) для обоих наборов моделей.

Данные:

https://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz

https://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz

https://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz

Какая модель, 1 или 2, оказалась точнее для первого здания (building_id = 0) ?

_______________________________________________________________________________________________________________________________

Импортируем библиотеки.

In [1]:
import pandas as pd
import numpy as np
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

Загружаем данные в датафреймы и фильтруем по первым 20 зданиям.

In [2]:
df_buildings = pd.read_csv('http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz')
df_weather = pd.read_csv('http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz').set_index(['timestamp', 'site_id'])
df_energy = pd.read_csv('http://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz')
df_energy = df_energy[df_energy['building_id'].isin(list(range(0, 20)))]

Соединяем датафреймы по индексам. Приводим дату к типу datetime, затем создаём колонку с часами. Создаём колонку поясняющую выходной день или нет. Фильтруем данные по показаниям счётчика, они должны быть больше нуля.

In [3]:
df = df_energy.merge(df_buildings, how = 'left', on = 'building_id')
df.set_index(['timestamp', 'site_id'], inplace = True)
df = df.merge(df_weather, left_index = True, right_index = True, how = 'left')
df.reset_index(inplace = True)
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['hours'] = df['timestamp'].apply(lambda x: x.hour)
df = df.drop(columns=["meter", "site_id", "floor_count"], axis=1)
dates_range = pd.date_range(start = '2015-12-31', end = '2017-01-01')
us_holidays = calendar().holidays(start = dates_range.min(), end = dates_range.max())
df['is_holiday'] = (df['timestamp'].isin(us_holidays)).astype('int8')
df = df[['timestamp', 'building_id', 'hours', 'air_temperature', 'dew_temperature', 'cloud_coverage', 'wind_speed',
        'precip_depth_1_hr', 'sea_level_pressure', 'is_holiday', 'meter_reading']][df.meter_reading > 0]

Функция уменьщения потребления памяти. Итерирует по колонкам, проверяет их тип и, если возможно, приводит их к типу, потребляющему меньше памяти.

In [4]:
def reduce_mem_usage (df):
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if str(col_type)[:5] == 'float':
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo('f2').min and c_max < np.finfo('f2').max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo('f4').min and c_max < np.finfo('f4').max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
        elif str(col_type)[:3] == 'int':
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo('i1').min and c_max < np.iinfo('i1').max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo('i2').min and c_max < np.iinfo('i2').max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo('i4').min and c_max < np.iinfo('i4').max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo('i8').min and c_max < np.iinfo('i8').max:
                df[col] = df[col].astype(np.int64)
        elif col == 'timestamp':
            df[col] = pd.to_datetime(df[col])
        elif str(col_type)[:8] != 'datetime':
            df[col] = df[col].astype('category')
    end_mem = df.memory_usage().sum() / 1024**2
    print(f'Потребление памяти меньше на {round(start_mem - end_mem, 2)} Мб (минус {round(100 * (start_mem - end_mem) / start_mem, 1)} %)')
    return df

df = reduce_mem_usage(df)

Потребление памяти меньше на 5.8 Мб (минус 62.9 %)


Заполняем пропуски интерполяцией данных.

In [5]:
df['precip_depth_1_hr'] = df['precip_depth_1_hr'].apply(lambda x: x if x > 0 else 0)
interpolate_columns = ['air_temperature', 'dew_temperature', 'cloud_coverage', 
                       'wind_speed', 'precip_depth_1_hr', 'sea_level_pressure']
for col in interpolate_columns:
    df[col] = df[col].interpolate(limit_direction='both', kind='cubic')
    
pd.set_option('use_inf_as_na', True)
for col in interpolate_columns:
    print(col, "Inf+NaN:", df[col].isnull().sum())

air_temperature Inf+NaN: 0
dew_temperature Inf+NaN: 0
cloud_coverage Inf+NaN: 0
wind_speed Inf+NaN: 0
precip_depth_1_hr Inf+NaN: 0
sea_level_pressure Inf+NaN: 0


Обогащаем данные индексами дней.

In [6]:
df.reset_index(inplace = True, drop = True)
df['day'] = df['timestamp'].apply(lambda x: str(x.day) if x.day < 7 else str(x.day % 7)).astype('category')
df = pd.get_dummies(df, columns = ['day'])
df.head()

,timestamp,building_id,hours,air_temperature,dew_temperature,cloud_coverage,wind_speed,precip_depth_1_hr,sea_level_pressure,is_holiday,meter_reading,day_0,day_1,day_2,day_3,day_4,day_5,day_6
0,2016-01-02 09:00:00,2,9,18.296875,16.703125,4.0,3.099609,0.0,1018.0,0,1.228516,0,0,1,0,0,0,0
1,2016-01-02 13:00:00,1,13,15.000000,13.898438,4.0,4.101562,0.0,1020.0,0,35.218750,0,0,1,0,0,0,0
2,2016-01-04 13:00:00,5,13,8.898438,3.900391,4.0,4.101562,0.0,1016.0,0,1.023438,0,0,0,0,1,0,0
3,2016-01-04 17:00:00,5,17,15.601562,-0.600098,0.0,4.101562,0.0,1017.0,0,0.682617,0,0,0,0,1,0,0
4,2016-01-04 17:00:00,11,17,15.601562,-0.600098,0.0,4.101562,0.0,1017.0,0,117.375000,0,0,0,0,1,0,0


Создаём списки с названием колонок для двух датасетов: с погодными данными и днями недель.

In [7]:
first_data = df[['building_id', 'hours', 'air_temperature', 'dew_temperature', 'cloud_coverage', 'wind_speed', 'sea_level_pressure', 'meter_reading']]
second_data = df[['building_id', 'hours', 'is_holiday', 'day_0', 'day_1','day_2','day_3','day_4','day_5','day_6', 'meter_reading']]

Разделяем данные на обучающие и проверочные.

In [8]:
first_data_train, first_data_test = train_test_split(first_data, test_size = 0.2, random_state = 42)
second_data_train, second_data_test = train_test_split(second_data, test_size = 0.2, random_state = 42)

Функция для обучения. Итерирует по индексу зданий и часам. Для каждого здания создаёт почасовую модель потребления электричества. Заносит полученные коэффициенты в список. Возвращает список.

In [9]:
def train_lr(df):
    models = []
    coeffs = []
    for build_id in sorted(df['building_id'].unique()):
        for hour in sorted(df.hours.unique()):
            X = df[df['building_id'] == build_id].drop(columns = ['building_id', 'meter_reading'])
            X = X[X['hours'] == hour].drop(columns = 'hours')
            y = df[df['building_id'] == build_id]
            y = y[y['hours'] == hour]['meter_reading']
            model = LinearRegression()
            model.fit(X, y)
            coeffs.append(model.coef_)
            coeffs[hour] = np.append(coeffs[hour], model.intercept_)
        models.append(coeffs)
        coeffs = []
    return models

Обучаем модели.

In [10]:
models_1 = train_lr(first_data_train)
models_2 = train_lr(second_data_train)

Проверяем качество модели с погодными данными.

In [11]:
def calculate_model_1(x):
    model = models_1[int(x.building_id)][int(x.hours)]
    meter_reading_log = np.log(x.meter_reading + 1)
    meter_reading_lr = np.log(1 + x.air_temperature * model[0] + 
        x.dew_temperature * model[1] + x.cloud_coverage * model[2] +
        x.wind_speed *  model[3] + x.sea_level_pressure * model[4] + model[5])
    x['meter_reading_lr_q'] = (meter_reading_log - meter_reading_lr)**2
    return x

first_data_test = first_data_test.apply(calculate_model_1, axis = 1, result_type = 'expand')
first_data_test_lr_rmsle = np.sqrt(first_data_test['meter_reading_lr_q'].sum() / len(first_data_test))
print (f'Качество модели линейной регрессии: {round(first_data_test_lr_rmsle, 6)}')

D:\Anaconda\envs\AI\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in log
  


Качество модели линейной регрессии: 0.241957


Проверяем качество модели с днями недели и выходными.

In [12]:
def calculate_model_2(x):
    model = models_2[int(x.building_id)][int(x.hours)]
    meter_reading_log = np.log(x.meter_reading + 1)
    meter_reading_lr = np.log(1 + x.is_holiday * model[0] + 
        x.day_0 * model[1] + x.day_1 * model[2] +
        x.day_2 *  model[3] + x.day_3 * model[4] + 
        x.day_4 * model[5] + x.day_5 * model[6] + 
        x.day_6 * model[7] + model[8])
    x['meter_reading_lr_q'] = (meter_reading_log - meter_reading_lr)**2
    return x

second_data_test = second_data_test.apply(calculate_model_2, axis = 1, result_type = 'expand')
second_data_test_lr_rmsle = np.sqrt(second_data_test['meter_reading_lr_q'].sum() / len(second_data_test))
print (f'Качество модели линейной регрессии: {round(second_data_test_lr_rmsle, 6)}')

Качество модели линейной регрессии: 0.295338


Сравниваем качество моделей по индексам зданий, выясняем какая модель эффективнее для каждого из 20 зданий.

In [15]:
for i in sorted(df['building_id'].unique()):
    x1 = first_data_test[first_data_test['building_id'] == i].apply(calculate_model_1, axis = 1, result_type = 'expand')
    x2 = second_data_test[second_data_test['building_id'] == i].apply(calculate_model_2, axis = 1, result_type = 'expand')
    x1_rmsle = np.sqrt(x1['meter_reading_lr_q'].sum() / len(x1))
    x2_rmsle = np.sqrt(x2['meter_reading_lr_q'].sum() / len(x2))
    if x1_rmsle < x2_rmsle:
        print(f'Первая модель для зданий с индексом {i} предпочтительнее ({round(x1_rmsle, 2)} < {round(x2_rmsle, 2)})')
    else:
        print(f'Вторая модель для зданий с индексом {i} предпочтительнее ({round(x1_rmsle, 2)} > {round(x2_rmsle, 2)})')

Первая модель для зданий с индексом 0 предпочтительнее (0.22 < 0.26)
Первая модель для зданий с индексом 1 предпочтительнее (0.3 < 0.38)


D:\Anaconda\envs\AI\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in log
  


Первая модель для зданий с индексом 2 предпочтительнее (0.41 < 0.57)
Первая модель для зданий с индексом 3 предпочтительнее (0.35 < 0.36)
Первая модель для зданий с индексом 4 предпочтительнее (0.22 < 0.23)
Первая модель для зданий с индексом 5 предпочтительнее (0.42 < 0.6)
Первая модель для зданий с индексом 6 предпочтительнее (0.21 < 0.31)
Первая модель для зданий с индексом 7 предпочтительнее (0.13 < 0.14)
Первая модель для зданий с индексом 8 предпочтительнее (0.1 < 0.15)
Первая модель для зданий с индексом 9 предпочтительнее (0.36 < 0.37)
Первая модель для зданий с индексом 10 предпочтительнее (0.16 < 0.17)
Первая модель для зданий с индексом 11 предпочтительнее (0.2 < 0.21)
Первая модель для зданий с индексом 12 предпочтительнее (0.28 < 0.31)
Первая модель для зданий с индексом 13 предпочтительнее (0.15 < 0.15)
Первая модель для зданий с индексом 14 предпочтительнее (0.11 < 0.11)
Первая модель для зданий с индексом 15 предпочтительнее (0.14 < 0.14)
Первая модель для зданий с инде

### Часть 4
##### Научимся экспортировать и импортировать промежуточные и конечные результаты, создавать ансамбли моделей.

### Описание задания

Загрузите данные и посчитайте модели линейной регрессии для 50 зданий по ансамблю регрессионных моделей: в первой модели весь оптимальный набор метеорологических данных, во второй - дни недели и праздники, в третьей - недели года, в четвертой - месяцы. Финальное значение показателя рассчитайте как взвешенное арифметическое показателей всех моделей, взяв веса для первой и второй модели как 3/8, а для третьей и четвертой - как 1/8.

Загрузите данные решения, посчитайте значение энергопотребления для требуемых дат для тех зданий, которые посчитаны в модели, и выгрузите результат в виде CSV-файла (submission.csv).

Данные:

http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz

http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz

http://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz

http://video.ittensive.com/machine-learning/ashrae/test.csv.gz

http://video.ittensive.com/machine-learning/ashrae/weather_test.csv.gz

Импортируем библиотеки.

In [1]:
import pandas as pd
import numpy as np
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

Загружаем данные.

In [2]:
buildings_metadata = pd.read_csv('http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz')
weather_train = pd.read_csv('http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz')
train_0 = pd.read_csv('http://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz')
test = pd.read_csv('http://video.ittensive.com/machine-learning/ashrae/test.csv.gz')
weather_test = pd.read_csv('http://video.ittensive.com/machine-learning/ashrae/weather_test.csv.gz')

Фильтруем данные по индексам первых 50 зданий. Объединяем обучающие данные по индексу. Приводим дату к типу datetime. Добавляем колонки с часами и выходными днями. Фильтруем датафрейм по показаниям счётчика, выбираем только нужные нам колонки. Логарифмируем показатель потребления энергии для уменьшения разброса данных.

In [3]:
weather_train.set_index(['timestamp', 'site_id'], inplace = True)
train_0 = train_0[train_0['building_id'].isin(list(range(0, 50)))]
df = train_0.merge(buildings_metadata, how = 'left', on = 'building_id')
df.set_index(['timestamp', 'site_id'], inplace = True)
df = df.merge(weather_train, left_index = True, right_index = True, how = 'left')
df.reset_index(inplace = True)
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['hours'] = df['timestamp'].apply(lambda x: x.hour)
df = df.drop(columns=["meter", "site_id", "floor_count"], axis=1)
dates_range = pd.date_range(start = '2015-12-31', end = '2017-01-01')
us_holidays = calendar().holidays(start = dates_range.min(), end = dates_range.max())
df['is_holiday'] = (df['timestamp'].isin(us_holidays)).astype('int8')
df = df[['timestamp', 'building_id', 'hours', 'air_temperature', 'dew_temperature', 'cloud_coverage', 'wind_speed',
         'sea_level_pressure', 'is_holiday', 'meter_reading']][df.meter_reading > 0]
df['meter_reading'] = df['meter_reading'].apply(lambda x: np.log(x) + 1) 
df.reset_index(inplace = True, drop = True)

Функция уменьщения потребления памяти. Итерирует по колонкам, проверяет их тип и, если возможно, приводит их к типу, потребляющему меньше памяти.

In [4]:
def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if str(col_type)[:5] == 'float':
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo('f2').min and c_max < np.finfo('f2').max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo('f4').min and c_max < np.finfo('f4').max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
        elif str(col_type)[:3] == 'int':
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo('i1').min and c_max < np.iinfo('i1').max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo('i2').min and c_max < np.iinfo('i2').max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo('i4').min and c_max < np.iinfo('i4').max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo('i8').min and c_max < np.iinfo('i8').max:
                df[col] = df[col].astype(np.int64)
        elif col == 'timestamp':
            df[col] = pd.to_datetime(df[col])
        elif str(col_type)[:8] != 'datetime':
            df[col] = df[col].astype('category')
    end_mem = df.memory_usage().sum() / 1024**2
    print(f'Потребление памяти меньше на {round(start_mem - end_mem, 2)} Мб (минус {round(100 * (start_mem - end_mem) / start_mem, 1)} %)')
    return df

df = reduce_mem_usage(df)

Потребление памяти меньше на 12.83 Мб (минус 68.5 %)


Заполняем пропуски в данных с помощью интерполяции.

In [5]:
interpolate_columns = ['air_temperature', 'dew_temperature', 'cloud_coverage', 
                       'wind_speed', 'sea_level_pressure']
for col in interpolate_columns:
    df[col] = df[col].interpolate(limit_direction='both', kind='cubic')
    
pd.set_option('use_inf_as_na', True)
for col in interpolate_columns:
    print(col, "Inf+NaN:", df[col].isnull().sum())

air_temperature Inf+NaN: 0
dew_temperature Inf+NaN: 0
cloud_coverage Inf+NaN: 0
wind_speed Inf+NaN: 0
sea_level_pressure Inf+NaN: 0


Создаём списки с названиями колонок для создания датафреймов с погодными данными, индексами дней, недель и месяцев. 

In [6]:
weather_features = df[['meter_reading', 'building_id', 'hours', 'air_temperature', 'dew_temperature', 'cloud_coverage',
                       'wind_speed', 'sea_level_pressure']]
days_features = df[['meter_reading', 'building_id', 'hours', 'timestamp', 'is_holiday']]
week_features = df[['meter_reading', 'building_id', 'hours', 'timestamp']]
month_features = df[['meter_reading', 'building_id', 'hours', 'timestamp']]

Создаём нужные нам датафреймы и обогащаем данные индексами дней, месяцев и недель.

In [7]:
days_features['day'] = days_features['timestamp'].apply(lambda x: str(x.day) if x.day < 7 else str(x.day % 7)).astype('category')
days_features = pd.get_dummies(days_features, columns = ['day'])
week_features['week'] = week_features['timestamp'].apply(lambda x: str(x.week)).astype('category')
week_features = pd.get_dummies(week_features, columns = ['week'])
month_features['month'] = month_features['timestamp'].apply(lambda x: str(x.month)).astype('category')
month_features = pd.get_dummies(month_features, columns = ['month'])

D:\Anaconda\envs\AI\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
D:\Anaconda\envs\AI\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Anaconda\envs\AI\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

In [8]:
days_features.drop('timestamp', axis = 1, inplace = True)
week_features.drop('timestamp', axis = 1, inplace = True)
month_features.drop('timestamp', axis = 1, inplace = True)

Функция для обучения. Итерирует по индексу зданий и часам. Для каждого здания создаёт почасовую модель потребления электричества. Заносит полученные коэффициенты в список. Возвращает список.

In [9]:
def train_lr(df):
    models = []
    coeffs = []
    for build_id in sorted(df['building_id'].unique()):
        for hour in sorted(df.hours.unique()):
            X = df[df['building_id'] == build_id].drop(columns = ['building_id', 'meter_reading'])
            X = X[X['hours'] == hour].drop(columns = 'hours')
            y = df[df['building_id'] == build_id]
            y = y[y['hours'] == hour]['meter_reading']
            model = LinearRegression(fit_intercept = False)
            model.fit(X, y)
            coeffs.append(model.coef_)
            coeffs[hour] = np.append(coeffs[hour], model.intercept_)
        models.append(coeffs)
        coeffs = []
    return models

Обучаем модели.

In [10]:
weather_model = train_lr(weather_features)
days_model = train_lr(days_features)
week_model = train_lr(week_features)
month_model = train_lr(month_features)

Заполняем пропуски в тестовых данных. Фильтруем их по индексу зданий. Объединяем с данными по зданиям.

In [11]:
for col in interpolate_columns:
    weather_test[col] = weather_test[col].interpolate(limit_direction='both', kind='cubic')
test = test[test.building_id < 50]
test = test[test.meter == 0]
test = pd.merge(left = test, right = buildings_metadata, how = 'left', left_on = 'building_id', right_on = 'building_id')

Дропаем лишние колонки. Приводим дату к типу datetime, обогащаем данные индексами дней, недель, месяцев, выходных дней и часов. Объединяем все тестовые данные: с погодными данными и данными по дням, часам и т.д.

In [12]:
test.drop(['row_id', 'primary_use', 'square_feet', 'year_built', 'floor_count'], axis = 1, inplace = True)
test.timestamp = pd.to_datetime(test.timestamp)
weather_test.timestamp = pd.to_datetime(weather_test.timestamp)
test['hours'] = test['timestamp'].apply(lambda x: x.hour)
test['is_holiday'] = (test['timestamp'].isin(us_holidays)).astype('int8')
test['day'] = test['timestamp'].apply(lambda x: str(x.day) if x.day < 7 else str(x.day % 7)).astype('category')
test = pd.get_dummies(test, columns = ['day'])
test['week'] = test['timestamp'].apply(lambda x: str(x.week)).astype('category')
test = pd.get_dummies(test, columns = ['week'])
test['month'] = test['timestamp'].apply(lambda x: str(x.month)).astype('category')
test = pd.get_dummies(test, columns = ['month'])
weather_test = weather_test.set_index(['timestamp', 'site_id'])
test = test.set_index(['timestamp', 'site_id'])

In [13]:
weather_test.drop(['precip_depth_1_hr', 'wind_direction'], axis = 1, inplace = True)
test = pd.merge(left = test, right = weather_test, how = 'left', left_index = True, right_index = True)
test = reduce_mem_usage(test)

Потребление памяти меньше на 42.6 Мб (минус 36.6 %)


Функция для вычисления качества ансамбля моделей линейной регрессии. Веса для моделей взяты как 1/8 для моделей с месяцами и неделями и 3/8 для моделей с погодными данными и днями. Финальный показатель рассчитан как как взвешенное арифметическое показателей всех моделей.

In [14]:
def calculate_test_data(df, weather_model, days_model, week_model, month_model):
    for row, i in enumerate(df.iterrows()):
        model_1 = weather_model[i[1][0]][i[1][2]]
        model_2 = days_model[i[1][0]][i[1][2]]
        model_3 = week_model[i[1][0]][i[1][2]]
        model_4 = month_model[i[1][0]][i[1][2]]
        total_lr = (3 / 8) * ((np.sum([c * v for c, v in zip(model_1[:-1], i[1][75:])])) 
                   + (np.sum([c * v for c, v in zip(model_2[:-1], i[1][3:11])]))) 
        + (1 / 8) * ((np.sum([c * v for c, v in zip(model_3[:-1], i[1][11:63])])) 
                   + (np.sum([c * v for c, v in zip(model_4[:-1], i[63:75])])))
        df['meter'][row] = np.exp(total_lr)
    return df

Рассчитываем показатель потребления электричества для тестовых данных и заносим результаты в csv файл.

In [16]:
test.meter = test.meter.astype('float')
result = calculate_test_data(test, weather_model, days_model, week_model, month_model)

D:\Anaconda\envs\AI\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [18]:
result = result['meter']
result.to_csv('submission.csv', index = False)